In [1]:
import argparse
import torch
import numpy as np
import os
import time
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import scipy.sparse as ss
import torch.optim as optim
import matplotlib.pyplot as plt
from dgtv.dgtv import *
import pickle
import logging
import sys
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler



In [5]:
from test_gtv import *
# from runcc import *

# !python test_gtv.py -w 512 -m model/GTV_19.pkl --stride 9 --multi 200 --image_path_test dataset/Test/Test_10
pathimage = "dataset/Test/Test_10/noisy/1.png"

# denoise_image(pathimage)
def load_gtv_model(model_path):
    # Tạo đối tượng OPT để cấu hình
    opt = OPT(
        batch_size=32,
        channels=1,
        lr=1e-4,
        momentum=0.9,
        u_max=1000,
        u_min=0.0001,
        cuda=True if torch.cuda.is_available() else False
    )

    # Thiết lập logger
    import logging
    logger = logging.getLogger("root")
    logger.addHandler(logging.StreamHandler(sys.stdout))
    opt.logger = logger
    opt.legacy = True

    # Chuẩn bị ma trận hỗ trợ
    supporting_matrix(opt)

    # Tạo đối tượng GTV
    gtv = GTV(width=16, cuda=opt.cuda, opt=opt)
    device = torch.device("cuda") if cuda else torch.device("cpu")
    # Load trọng số đã được đào tạo
    gtv.load_state_dict(torch.load(model_path, map_location=device))
    # gtv.cuda()
    width = gtv.opt.width
    opt.width = width
    opt = gtv.opt

    return gtv



In [10]:
from train_gtv import *

In [11]:
gtv_model = load_gtv_model("model/GTV_99.pkl")

In [22]:
opt = OPT(
    batch_size=32,
    channels=1,
    lr=1e-4,
    momentum=0.9,
    u_max=1000,
    u_min=0.0001,
    cuda=True if torch.cuda.is_available() else False
    )
import logging
logger = logging.getLogger("root")
logger.addHandler(logging.StreamHandler(sys.stdout))
opt.logger = logger
opt.legacy = True

# Chuẩn bị ma trận hỗ trợ
supporting_matrix(opt)
logger.info("GTV evaluation")

def denoise_image(
    inp,
    gtv,
    stride=16, #ảnh hưởng đến tốc độ denoise
    width=None,
    prefix="img1",
    verbose=0,
    opt=opt,
    approx=False,
    args=None,
    logger=logger
):

    sample = cv2.imread(inp)
    if width is None:
        width = sample.shape[0]
    else:
        sample = cv2.resize(sample, (width, width))
        
    sample = cv2.cvtColor(sample, cv2.COLOR_RGB2GRAY)
    sample = np.expand_dims(sample, axis=2)
    sample = sample.transpose((2, 0, 1))
    shape = sample.shape

    sample = torch.from_numpy(sample)

    cuda = True if torch.cuda.is_available() else False

    dtype = torch.cuda.FloatTensor if cuda else torch.FloatTensor


    tstart = time.time()
    T1 = sample

    T1 = torch.nn.functional.pad(T1, (0, stride, 0, stride), mode="constant", value=0)
    shapex = T1.shape
    T2 = (
        torch.from_numpy(T1.detach().numpy().transpose(1, 2, 0))
        .unfold(0, opt.width, stride)
        .unfold(1, opt.width, stride)
    ).type(dtype)
    T2 = T2.contiguous()

    # MAX_PATCH = args.multi
    MAX_PATCH = 500
    oT2s0 = T2.shape[0]
    T2 = T2.view(-1, opt.channels, opt.width, opt.width)
    dummy = torch.zeros(T2.shape).type(dtype)
    logger.info("{0}".format(T2.shape))
    with torch.no_grad():
        for ii, i in enumerate(range(0, T2.shape[0], MAX_PATCH)):
            P = gtv.predict(
                T2[i : (i + MAX_PATCH), :, :, :].float().contiguous().type(dtype),
            )
            dummy[i : (i + MAX_PATCH)] = P
    dummy = dummy.view(oT2s0, -1, opt.channels, opt.width, opt.width)
    dummy = dummy.cpu()
    if verbose:
        logger.info("Prediction time: {0}".format(time.time() - tstart))
    else:
        logger.info("Prediction time: {0}".format(time.time() - tstart))

    dummy = (
        patch_merge(dummy, stride=stride, shape=shapex, shapeorg=shape).detach().numpy()
    )

    ds = np.array(dummy).copy()
    d = np.minimum(np.maximum(ds, 0), 255)

    logger.info("RANGE: {0} - {1}".format(d.min(), d.max()))
    d = d.transpose(1, 2, 0) / 255
    d = d[:,:,0]
    if 0:
        opath = args.output
    else:
        filename = inp.split("/")[-1]
        opath = resroot + "/{0}_{1}".format(prefix, filename)
        opath = opath[:-3] + "png"
    d = np.minimum(np.maximum(d, 0), 1)
    plt.imsave(opath, d, cmap='gray')

    return d



In [24]:
imr="dataset/Test/Test_5/noisy/4.png"
imp = "dataset/Bao/63.png"
denoise_image(imr,gtv_model, stride = 8, prefix= "img1_")

array([[0.00746012, 0.00746012, 0.00746012, ..., 0.00679831, 0.00679709,
        0.00679409],
       [0.00746012, 0.00746012, 0.00746012, ..., 0.00679804, 0.00679678,
        0.00679422],
       [0.00746012, 0.00746012, 0.00746012, ..., 0.00679777, 0.00679667,
        0.00679416],
       ...,
       [0.00353859, 0.00353859, 0.00353859, ..., 0.00461285, 0.00461272,
        0.00461249],
       [0.00353859, 0.00353858, 0.00353858, ..., 0.0046123 , 0.00461213,
        0.00461176],
       [0.00353858, 0.00353858, 0.00353858, ..., 0.00463835, 0.00461193,
        0.00461126]], dtype=float32)

In [14]:
# Đọc ảnh nguồn
sample = cv2.imread(imr)

# Kiểm tra ảnh có tồn tại không
if sample is None:
    print(f"Không thể đọc ảnh từ đường dẫn: {imr}")
else:
    # In kích thước của ảnh nguồn
    print(f"Kích thước của ảnh nguồn: {sample.shape}")

    # Tiếp tục thực hiện các bước khác (resize, chuyển đổi màu, v.v.)

Không thể đọc ảnh từ đường dẫn: /content/PBL6-v1/deepgtv/dataset/Test/Test_10/noisy/4.png
